In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Scaling
from sklearn.preprocessing import MinMaxScaler

 #Correlation Heatmap
from matplotlib.colors import LinearSegmentedColormap

#Statistical Test
from scipy import stats
from sklearn.impute import SimpleImputer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None)






In [23]:
# You can download the data in the source that is linked above the table of contents

# Read in the data
X_val_encoded = pd.read_csv('../project_data/X_val_encoded.csv', delimiter=',', index_col=0)
X_train_encoded = pd.read_csv('../project_data/X_train_encoded.csv', delimiter=',', index_col=0)

y_train = pd.read_csv('../project_data/y_train.csv',delimiter=',', index_col=0)
y_val= pd.read_csv('../project_data/y_val.csv', delimiter=',', index_col=0)

X_test_encoded = pd.read_csv('../project_data/X_test_encoded.csv',index_col=0)

In [ ]:
#NEW: just to check the shape
print(X_train_encoded.shape[0])
print(y_train.shape[0])

382537
382537


In [27]:
claim_injury_type_mapping = {
    '4. TEMPORARY': 4,
    '2. NON-COMP': 2,
    '5. PPD SCH LOSS': 5,
    '3. MED ONLY': 3,
    '6. PPD NSL': 6,
    '1. CANCELLED': 1,
    '8. DEATH': 8,
    '7. PTD': 7
}

y_train_encoded = y_train['Claim Injury Type'].map(claim_injury_type_mapping)
y_val_encoded = y_val['Claim Injury Type'].map(claim_injury_type_mapping)
#NEW
y_train_encoded = pd.DataFrame(y_train_encoded, columns=['Claim Injury Type'])
y_val_encoded = pd.DataFrame(y_val_encoded, columns=['Claim Injury Type'])

In [29]:
num_columns = ['Age at Injury', 'Average Weekly Wage', 'Birth Year', 'IME-4 Count', 'Number of Dependents', 'Days_between_Acc_Assembyl']

cat_columns = ['Alternative Dispute Resolution', 'Attorney/Representative', 'C-2 Date', 'C-3 Date', 'COVID-19 Indicator', 'Gender','First Hearing Date',
               'CarrierGroup_Self-insured Private Entity', 'CarrierGroup_Self-insured Public Entity', 'CarrierGroup_Special Funds',
               'CarrierGroup_State Insurance Fund', 'Industry Code_encoded_5. PPD SCH LOSS', 'Industry Code_encoded_2. NON-COMP', 
               'Industry Code_encoded_3. MED ONLY', 'Industry Code_encoded_4. TEMPORARY', 'Industry Code_encoded_1. CANCELLED', 
               'Industry Code_encoded_8. DEATH', 'Industry Code_encoded_6. PPD NSL', 'Industry Code_encoded_7. PTD',
               'Injury_Cause_Category_encoded_5. PPD SCH LOSS', 'Injury_Cause_Category_encoded_2. NON-COMP', 
               'Injury_Cause_Category_encoded_3. MED ONLY', 'Injury_Cause_Category_encoded_4. TEMPORARY', 
               'Injury_Cause_Category_encoded_1. CANCELLED', 'Injury_Cause_Category_encoded_8. DEATH','Injury_Cause_Category_encoded_6. PPD NSL',
                'Injury_Cause_Category_encoded_7. PTD', 'Nature_Injury_Occupational', 'Nature_Injury_Specific', 'Nature_Injury_Unknown',
                'Part_Body_Category_encoded_5. PPD SCH LOSS', 'Part_Body_Category_encoded_2. NON-COMP', 'Part_Body_Category_encoded_3. MED ONLY',
                'Part_Body_Category_encoded_4. TEMPORARY', 'Part_Body_Category_encoded_1. CANCELLED', 'Part_Body_Category_encoded_8. DEATH',
                'Part_Body_Category_encoded_6. PPD NSL', 'Part_Body_Category_encoded_7. PTD']


# Create subsets
X_train_num = X_train_encoded[num_columns]
X_train_cat = X_train_encoded[cat_columns]

X_val_num = X_val_encoded[num_columns]
X_val_cat = X_val_encoded[cat_columns]

X_test_num=X_test_encoded[num_columns]
X_test_cat=X_test_encoded[cat_columns]

In [30]:
scaler = MinMaxScaler().fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)

# Convert the array to a pandas dataframe
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train_encoded.index)
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val_encoded.index)

X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns).set_index(X_test_encoded.index)

In [31]:
# Remerge the numerical and categorical features
X_train = pd.concat([X_train_num[['IME-4 Count']], X_train_cat], axis=1)
X_val = pd.concat([X_val_num[['IME-4 Count']], X_val_cat], axis=1)
X_test =pd.concat([X_test_num[['IME-4 Count']], X_test_cat], axis=1)

In [ ]:
def avg_score(model):
    score_train = []
    score_val = []
    timer = []
    f1_score_val = []
    f1_score_train = []
    
    for i in range(10):
        begin = time.perf_counter() # start counting time
        model.fit(X_train, y_train) # fit your model to your training data
        end = time.perf_counter() # stop counting time
        
        value_train = model.score(X_train, y_train) # mean accuracy for train
        value_val = model.score(X_val, y_val) # mean accuracy for validation
        score_train.append(value_train) # append the mean accuracy in train to your list score_train
        score_val.append(value_val) # append the mean accuracy in validation to your list score_val
        timer.append(end-begin) # append the time to your list timer
        f1_score_val.append(f1_score(model.predict(X_val), y_val))
        f1_score_train.append(f1_score(model.predict(X_train), y_train))
        
    
    avg_time = round(np.mean(timer),3) # check the mean value of training time for your 10 models 
    avg_train = round(np.mean(score_train),3) # check the mean accuracy in train for your 10 models
    avg_val = round(np.mean(score_val),3) # check the mean accuracy in validation for your 10 models
    std_time = round(np.std(timer),2) # check the standard deviation of training time for your 10 models
    std_train = round(np.std(score_train),2) # check the standard deviation of the mean accuracy in train for your 10 models
    std_val = round(np.std(score_val),2) # check the standard deviation of the mean accuracy in validation for your 10 models
    avg_f1_val = round(np.mean(f1_score_val),3)
    avg_f1_train = round(np.mean(f1_score_train),3)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_val) + '+/-' + str(std_val), str(avg_f1_train), str(avg_f1_val)

def show_results(df, models):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    for model in models:
        # obtain the results provided by avg_score
        time, avg_train, avg_val, avg_f1_train, avg_f1_val = avg_score(model)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_val, avg_f1_train, avg_f1_val
        count+=1
    return df

In [ ]:
# Importações e conjuntos de dados permanecem fora da função
from sklearn.model_selection import StratifiedKFold
import time
import numpy as np

# Função para cálculo do score médio com validação cruzada
def avg_score(model, X_train, y_train):
    # Aplicar StratifiedKFold
    skf = StratifiedKFold(n_splits=10)
    
    # Listas para armazenar os resultados
    score_train = []
    score_val = []
    timer = []
    n_iter = []
    
    for train_index, val_index in skf.split(X_train, y_train):
        # Separar os dados em treino e validação
        X_t, X_v = X_train.iloc[train_index], X_train.iloc[val_index]
        y_t, y_v = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # Iniciar contagem de tempo
        begin = time.perf_counter()
        
        # Ajustar o modelo
        model.fit(X_t, y_t)
        
        # Fim da contagem de tempo
        end = time.perf_counter()
        
        # Avaliar o modelo
        value_train = model.score(X_t, y_t)
        value_val = model.score(X_v, y_v)
        
        # Armazenar resultados
        score_train.append(value_train)
        score_val.append(value_val)
        timer.append(end - begin)
        n_iter.append(getattr(model, 'n_iter_', 0))  # Evitar erro se o modelo não tiver `n_iter_`
    
    # Calcular médias e desvios padrão
    avg_time = round(np.mean(timer), 3)
    avg_train = round(np.mean(score_train), 3)
    avg_val = round(np.mean(score_val), 3)
    std_time = round(np.std(timer), 2)
    std_train = round(np.std(score_train), 2)
    std_val = round(np.std(score_val), 2)
    avg_iter = round(np.mean(n_iter), 1)
    std_iter = round(np.std(n_iter), 1)
    
    return (f"{avg_time} +/- {std_time}", f"{avg_train} +/- {std_train}", 
            f"{avg_val} +/- {std_val}", f"{avg_iter} +/- {std_iter}")

# Criar o dataframe de resultados fora da função
results_df = pd.DataFrame(columns=['Time', 'Train Accuracy', 'Validation Accuracy', 'Iterations'], index=['LogReg', 'RandomForest'])

# Avaliar os modelos fora da função
model_1 = LogisticRegression(max_iter=1000)
model_2 = RandomForestClassifier(n_estimators=100)

time1, train1, val1, iter1 = avg_score(model_1, X_train, y_train_encoded['Claim Injury Type'])
results_df.loc['LogReg'] = [time1, train1, val1, iter1]

time2, train2, val2, iter2 = avg_score(model_2, X_train, y_train_encoded['Claim Injury Type'])
results_df.loc['RandomForest'] = [time2, train2, val2, iter2]

# Exibir os resultados
print(results_df)


def avg_score(model):
    # apply kfold
    skf = StratifiedKFold(n_splits=10)
    # create lists to store the results from the different models 
    score_train = []
    score_val = []
    timer = []
    n_iter = []
    
    for train_index, val_index in skf.split(X,y):
        # get the indexes of the observations assigned for each partition
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        # start counting time
        begin = time.perf_counter()
        
        

 #SCALING, e toda a pre processing       
        
        
        
        # fit the model to the data
        model.fit(X_train, y_train)
        # finish counting time
        end = time.perf_counter()
        # check the mean accuracy for the train
        value_train = model.score(X_train, y_train)
        # check the mean accuracy for the validation
        value_val = model.score(X_val,y_val)
        # append the accuracies, the time and the number of iterations in the corresponding list
        score_train.append(value_train)
        score_val.append(value_val)
        timer.append(end-begin)
        n_iter.append(model.n_iter_)
    # calculate the average and the std for each measure (accuracy, time and number of iterations)
    avg_time = round(np.mean(timer),3)
    avg_train = round(np.mean(score_train),3)
    avg_val = round(np.mean(score_val),3)
    std_time = round(np.std(timer),2)
    std_train = round(np.std(score_train),2)
    std_val = round(np.std(score_val),2)
    avg_iter = round(np.mean(n_iter),1)
    std_iter = round(np.std(n_iter),1)
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_val) + '+/-' + str(std_val), str(avg_iter) + '+/-' + str(std_iter)

def show_results(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    # for each model passed as argument
    for arg in args:
        # obtain the results provided by avg_score
        time, avg_train, avg_val, avg_iter = avg_score(arg)
        # store the results in the right row
        df.iloc[count] = time, avg_train, avg_val, avg_iter
        count+=1
    return df

Modelos Tradicionais de Machine Learning
Logistic Regression Multiclasse

Funcionamento: Extensão da regressão logística binária para prever várias categorias com técnicas como One-vs-Rest (OvR) ou Softmax.
Vantagens: Simples de entender, eficiente em datasets pequenos.
Limitações: Não lida bem com relações complexas entre as variáveis ou entre os rótulos.
Escalabilidade: Precisa de dados normalizados.
Decision Trees (DT)

Funcionamento: Constrói árvores de decisão com base em critérios de impureza (como Gini ou Entropia).
Vantagens: Funciona com Ordinal Encoding, não exige normalização.
Limitações: Tende a sobreajustar em datasets pequenos.
Random Forest (RF)

Funcionamento: Ensemble de árvores de decisão para reduzir o sobreajuste.
Vantagens: Robusto contra overfitting, lida bem com categorical encoding.
Limitações: Pode ser mais lento em grandes datasets.
Gradient Boosting (e.g., XGBoost, LightGBM)

Funcionamento: Constrói modelos iterativamente, corrigindo erros das previsões anteriores.
Vantagens: Excelente para dados tabulares com muitas categorias.
Limitações: Exige Ordinal Encoding ou One-Hot Encoding.
Modelos Baseados em Redes Neurais
Multilayer Perceptron (MLP)

Funcionamento: Redes totalmente conectadas com ativação Softmax para saída multicategoria.
Vantagens: Flexível, pode capturar relações complexas.
Limitações: Exige grande volume de dados, sensível à escala.
Convolutional Neural Networks (CNNs)

Funcionamento: Modelos baseados em convolução, ideais para imagens ou séries temporais.
Vantagens: Excelente para dados espaciais.
Limitações: Ineficiente para dados tabulares ou não estruturados.
Recurrent Neural Networks (RNNs) e Transformers

Funcionamento: Ideais para dados sequenciais (texto ou séries temporais).
Vantagens: Capturam dependências temporais.
Limitações: Mais complexos de treinar.
Modelos de Similaridade e Vizinho Mais Próximo
K-Nearest Neighbors (KNN)
Funcionamento: Classifica com base nos rótulos dos vizinhos mais próximos.
Vantagens: Simples de implementar, não precisa de treinamento explícito.
Limitações: Sensível à escala (precisa de normalização).
Modelos Estatísticos
Naive Bayes
Funcionamento: Calcula probabilidades assumindo independência entre atributos.
Vantagens: Rápido, eficiente com dados textuais (TF-IDF).
Limitações: Assume independência entre variáveis (o que nem sempre é verdade).
Considerações sobre Pré-processamento
Encoding de Variáveis Categóricas:
Ordinal Encoding: Adequado para modelos baseados em árvores, mas não para métodos baseados em distância como KNN.
One-Hot Encoding: Necessário para redes neurais e modelos lineares (Logistic Regression, SVM).
Normalização:
Essencial para algoritmos baseados em distância (KNN, SVM) e redes neurais.
Dados Desequilibrados:
Métodos como Oversampling (SMOTE) ou ajuste de pesos nas classes podem ser usados.